In [ ]:
import numpy as np
import random
from collections import Counter
import time as tm
import pickle
import warnings
import os

def my_fit(words, verbose=False):
    dt = Dictionary(words)
    return dt

def my_predict(dt, bigrams, min_bigrams=5, max_words=5):
    word_counter = Counter()
    for bigram in bigrams:
        if bigram in dt.bigram_word_map:
            word_counter.update(dt.bigram_word_map[bigram])

    # Filter words based on the minimum and maximum number of bigrams
    filtered_words = [(word, count) for word, count in word_counter.items() if min_bigrams <= count <= max_words]

    # Sort filtered words by count in descending order
    filtered_words.sort(key=lambda x: x[1], reverse=True)

    # Select up to max_words most common words
    most_common_words = [word for word, count in filtered_words[:max_words]]

    return most_common_words

class Dictionary:
    def __init__(self, words):
        self.words = words
        self.bigram_word_map = self.create_bigram_word_map(words)

    def create_bigram_word_map(self, words):
        bigram_word_map = {}
        for word in words:
            bigrams = self.get_bigrams(word)
            for bigram in bigrams:
                if bigram not in bigram_word_map:
                    bigram_word_map[bigram] = []
                bigram_word_map[bigram].append(word)
        return bigram_word_map

    def get_bigrams(self, word, lim=5):
        bg = map(''.join, zip(word, word[1:]))
        bg = sorted(set(bg))
        return list(bg)[:lim]

    def predict(self, bigrams):
        word_counter = Counter()
        for bigram in bigrams:
            if bigram in self.bigram_word_map:
                word_counter.update(self.bigram_word_map[bigram])
        most_common_words = [word for word, count in word_counter.most_common(5)]
        return most_common_words

class Tree:
    def __init__(self, min_leaf_size=1, max_depth=1):
        self.root = None
        self.words = None
        self.min_leaf_size = min_leaf_size
        self.max_depth = max_depth
        self.dictionary = None  # Add a dictionary attribute

    def fit(self, words, verbose=False):
        self.words = words
        self.dictionary = my_fit(words, verbose)  # Initialize the dictionary
        self.root = Node(depth=0, parent=None, dictionary=self.dictionary)
        if verbose:
            print("root")
            print("└───", end='')
        # The root is trained with all the words
        self.root.fit(all_words=self.words, my_words_idx=np.arange(len(self.words)),
                      min_leaf_size=self.min_leaf_size, max_depth=self.max_depth, verbose=verbose)

    def predict(self, bg):
        return self.dictionary.predict(bg)

class Node:
    def __init__(self, depth, parent, dictionary):
        self.depth = depth
        self.parent = parent
        self.all_words = None
        self.my_words_idx = None
        self.children = {}
        self.is_leaf = True
        self.query = None
        self.history = []
        self.dictionary = dictionary  # Pass the dictionary to the node

    def get_query(self):
        return self.query

    def get_child(self, response):
        if self.is_leaf:
            print("Why is a leaf node being asked to produce a child? Melbo should look into this!!")
            child = self
        else:
            if response not in self.children:
                print(f"Unknown response {response} -- need to fix the model")
                response = list(self.children.keys())[0]

            child = self.children[response]

        return child

    def process_leaf(self, all_words, my_words_idx, history, verbose):
        self.my_words_idx = my_words_idx

    def get_bigrams(self, word, lim=5):
        bg = map(''.join, zip(word, word[1:]))
        bg = sorted(set(bg))
        return tuple(bg)[:lim]

    def get_random_bigram(self):
        return chr(ord('a') + random.randint(0, 25)) + chr(ord('a') + random.randint(0, 25))

    def process_node(self, all_words, my_words_idx, history, verbose):
        query = self.get_random_bigram()

        if (query in history) and verbose:
            print(f"Warning: bigram being re-used -- bigram {query} has already been used by an ancestor node as the splitting criterion. This is suboptimal!")

        split_dict = {}
        split_dict[True] = []
        split_dict[False] = []

        for idx in my_words_idx:
            bg_list = self.get_bigrams(all_words[idx])
            split_dict[query in bg_list].append(idx)

        if len(split_dict.items()) < 2 and verbose:
            print("Warning: did not make any meaningful split with this query!")

        return query, split_dict

    def fit(self, all_words, my_words_idx, min_leaf_size, max_depth, fmt_str="    ", verbose=False):
        self.all_words = all_words
        self.my_words_idx = my_words_idx

        if len(my_words_idx) <= min_leaf_size or self.depth >= max_depth:
            self.is_leaf = True
            self.process_leaf(self.all_words, self.my_words_idx, self.history, verbose)
            if verbose:
                print('█')
        else:
            self.is_leaf = False
            query, split_dict = self.process_node(self.all_words, self.my_words_idx, self.history, verbose)

            if verbose:
                print(query)

            for i, (response, split) in enumerate(split_dict.items()):
                if verbose:
                    if i == len(split_dict) - 1:
                        print(fmt_str + "└───", end='')
                        fmt_str += "    "
                    else:
                        print(fmt_str + "├───", end='')
                        fmt_str += "│   "

                self.children[response] = Node(depth=self.depth + 1, parent=self, dictionary=self.dictionary)
                history = self.history.copy()
                history.append(query)
                self.children[response].history = history

                self.children[response].fit(self.all_words, split, min_leaf_size, max_depth, fmt_str, verbose)

In [ ]:
with open( "dict_secret", 'r' ) as f:
	words = f.read().split( '\n' )[:-1]		# Omit the last line since it is empty
	num_words = len( words )

In [ ]:
n_trials = 5

t_train = 0
m_size = 0
t_test = 0
prec = 0

In [ ]:
def get_bigrams( word, lim = None ):
  # Get all bigrams
  bg = map( ''.join, list( zip( word, word[1:] ) ) )
  # Remove duplicates and sort them
  bg = sorted( set( bg ) )
  # Make them into an immutable tuple and retain only the first few
  return tuple( bg )[:lim]


In [ ]:
lim_bg = 5
lim_out = 5

for t in range(n_trials):
    tic = tm.perf_counter()
    model = my_fit(words)
    toc = tm.perf_counter()
    t_train += toc - tic

    with open(f"model_dump_{t}.pkl", "wb") as outfile:
        pickle.dump(model, outfile, protocol=pickle.HIGHEST_PROTOCOL)

    m_size += os.path.getsize(f"model_dump_{t}.pkl")

    tic = tm.perf_counter()

    for (i, word) in enumerate(words):
        bg = get_bigrams(word, lim=lim_bg)
        guess_list = my_predict(model, bg)

        # Do not send long guess lists -- they will result in lower marks
        guess_len = len(guess_list)
        # Ignore all but the first 5 guesses
        guess_list = guess_list[:lim_out]

        # Notice that if 10 guesses are made, one of which is correct,
        # score goes up by 1/10 even though only first 5 guesses are considered
        # Thus, it is never beneficial to send more than 5 guesses
        if word in guess_list:
            prec += 1 / guess_len

    toc = tm.perf_counter()
    t_test += toc - tic




In [ ]:
t_train /= n_trials
m_size /= n_trials
t_test /= n_trials
prec /= ( n_trials * num_words )

print( t_train, m_size, prec, t_test )